**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [4]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Dropout

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?


The function _act_ takes the current state *s* as input and returns a possible action. 

- At train time : With a probability $\epsilon$, the agent chooses an action uniformly at random (exploration of the environment). Otherwise (with a probability $1-\epsilon$) it will choose the action maximizing the expected cumulative reward from his current state $s$ (greedy choice).

- At test time : The agent follows the learnt policy (greedy policy).

- $\epsilon$ models a tradeoff between exploration of the environment and exploitation of the current knowledge. It is essential because at the beginning the agent has a very limited knowledge of the environment, and $\epsilon$ ensures that it keeps exploring and increasing that knowledge.



***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train= 40 # set small when debugging
epochs_test=  30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

- The array "position" gives to the rat the information of where it can go among the cells it sees (0 if the cell is accessible, -1 if not accessible i.e: the boundaries, and 1 if the rat is already in the cell).

- The array "board" gives to the rat the rewards present in each cell he sees (0.5 if there is a cheese in the cell, -1 if there is poison, 0 if there is nothing). 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        action = np.random.randint(0, self.n_action, size=1)[0]
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        ##### FILL IN HERE
        state = env.reset()
        win = 0
        lose = 0
        game_over = False
        
        while not game_over:
            action = agent.act(state)

            prev_state = state
            state, reward, game_over = env.act(action)
            
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [11]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 3.0/7.0. Average score (-4.0)
Win/lose count 13.0/10.0. Average score (-0.5)
Win/lose count 6.0/18.0. Average score (-4.333333333333333)
Win/lose count 8.5/11.0. Average score (-3.875)
Win/lose count 6.5/11.0. Average score (-4.0)
Win/lose count 4.0/4.0. Average score (-3.3333333333333335)
Win/lose count 8.5/21.0. Average score (-4.642857142857143)
Win/lose count 4.0/11.0. Average score (-4.9375)
Win/lose count 7.5/18.0. Average score (-5.555555555555555)
Win/lose count 6.5/15.0. Average score (-5.85)
Win/lose count 15.5/13.0. Average score (-5.090909090909091)
Win/lose count 8.5/20.0. Average score (-5.625)
Win/lose count 9.5/18.0. Average score (-5.846153846153846)
Win/lose count 14.5/16.0. Average score (-5.535714285714286)
Win/lose count 5.5/12.0. Average score (-5.6)
Win/lose count 9.0/12.0. Average score (-5.4375)
Win/lose count 10.0/19.0. Average score (-5.647058823529412)
Win/lose count 8.0/8.0. Average score (-5.333333333333333)
Win/lose count 11.0/16.0. Average

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




 According to the definition:
 \begin{align*}
    Q^\pi (s,a) &= E_{p^\pi} [\sum_{t=0}^\infty \gamma^t r(s_t, a_t)|s_0 = s, a_0 = a] \\
    &= E_{p^\pi} [r_0 + \gamma \sum_{t=0}^\infty \gamma^t r(s_{t+1}, a_{t+1})|s_0 = s, a_0 = a]
  \end{align*}
  
 The first term can be written as \begin{align*}
    E_{p^\pi} [r_0|s_0 = s, a_0 = a] &= \sum_{s'} p(s'|s, a) r(s, a) \\
  \end{align*}
  
 The second term can be written as 

\begin{align*}
    E_{p^\pi} [\gamma \sum_{t=0}^\infty \gamma^t r(s_{t+1}, a_{t+1})|s_0 = s, a_0 = a] &= \sum_{s'} p(s'|s, a) \gamma E_{p^\pi} [\sum_{t=0}^\infty \gamma^t r(s_{t+1}, a_{t+1}) | s_1 = s']\\
  \end{align*}
  
 Thus the sum is 
  \begin{align*}
    Q^\pi (s,a) &=  \sum_{s'} p(s'|s, a) \Big (r(s, a) + \gamma \sum_{a'} E_{p^\pi} [\sum_{t=0}^\infty \gamma^t r(s_{t+1}, a_{t+1}) | s_1 = s', a_1 = a'] \Big ) \\
    &= \sum_{s'} p(s'|s, a) \Big (r(s, a) + \gamma Q^\pi (s', a') \Big ) \\
    &= E_{(s',a') \thicksim p(\cdot|s, a)} [r(s,a) + \gamma Q^\pi (s', a')]
  \end{align*}
  
 Given that the optimal policy $\pi^* = \mbox{argmax}_{a} = Q^* (s,a)$ exists, we have

 \begin{align*}
    Q^*(s,a) &= \max_{\pi}Q^\pi(s,a) = Q^{\pi^*}(s,a) \\
    &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
  \end{align*}
  
 What we want to minimize is the difference between the Q-learning target $Q^*(s,a, \theta)$ and the Q-learning prediction $Q(s,a, \theta)$, which can be formulated into the following least-square problem:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>self.max_memory-1:
            self.memory.pop(0)
        self.memory.append(m)
                            
    def random_access(self):
        rnd = np.random.randint(0,len(self.memory))
        return self.memory[rnd]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        action_values = self.model.predict(s.reshape(1, 5, 5, self.n_state))
        return np.argmax(action_values)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            state, new_state, action, reward, game_over_ = self.memory.random_access()                        
            input_states[i] = state
            target_q[i] = self.model.predict(state.reshape(1, 5, 5, self.n_state))
            
            if game_over_:
                ######## FILL IN
                target_q[i][action] = reward
            else:
                ######## FILL IN
                target_q[i][action] = reward + self.discount * np.max(self.model.predict(new_state.reshape(1, 5, 5, self.n_state)))
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential([
            Flatten(input_shape=(5, 5, self.n_state)),
            Dense(32, activation="relu"),
            Dense(16, activation="relu"),
            Dense(4)])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))













Epoch 000/040 | Loss 0.0034 | Win/lose count 2.0/4.0 (-2.0)
Epoch 001/040 | Loss 0.0102 | Win/lose count 1.0/5.0 (-4.0)
Epoch 002/040 | Loss 0.0168 | Win/lose count 4.0/6.0 (-2.0)
Epoch 003/040 | Loss 0.0024 | Win/lose count 4.5/2.0 (2.5)
Epoch 004/040 | Loss 0.0075 | Win/lose count 4.5/9.0 (-4.5)
Epoch 005/040 | Loss 0.0029 | Win/lose count 1.0/5.0 (-4.0)
Epoch 006/040 | Loss 0.0182 | Win/lose count 3.0/4.0 (-1.0)
Epoch 007/040 | Loss 0.0160 | Win/lose count 4.0/6.0 (-2.0)
Epoch 008/040 | Loss 0.0022 | Win/lose count 1.5/1.0 (0.5)
Epoch 009/040 | Loss 0.0101 | Win/lose count 5.0/7.0 (-2.0)
Epoch 010/040 | Loss 0.0101 | Win/lose count 4.0/6.0 (-2.0)
Epoch 011/040 | Loss 0.0016 | Win/lose count 1.0/1.0 (0.0)
Epoch 012/040 | Loss 0.0156 | Win/lose count 5.5/1.0 (4.5)
Epoch 013/040 | Loss 0.0063 | Win/lose count 5.0/3.0 (2.0)
Epoch 014/040 | Loss 0.0109 | Win/lose count 1.5/3.0 (-1.5)
Epoch 015/040 | Loss 0.0072 | Win/lose count 4.5/1.0 (3.5)
Epoch 016/040 | Loss 0.0059 | Win/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential([
            Conv2D(64, (3,3), activation='relu', padding="same",input_shape=(5,5,self.n_state)),
            Conv2D(32, (3,3), activation='relu',padding="same"),
            Conv2D(32, (3,3), activation='relu',padding="same"),
            Flatten(),
            Dense(32),
            Dropout(0.2),
            Dense(4) 
        ])

        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [26]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/040 | Loss 0.0013 | Win/lose count 2.0/8.0 (-6.0)
Epoch 001/040 | Loss 0.0032 | Win/lose count 3.0/2.0 (1.0)
Epoch 002/040 | Loss 0.0040 | Win/lose count 1.0/5.0 (-4.0)
Epoch 003/040 | Loss 0.0017 | Win/lose count 4.0/6.0 (-2.0)
Epoch 004/040 | Loss 0.0047 | Win/lose count 4.0/3.0 (1.0)
Epoch 005/040 | Loss 0.0013 | Win/lose count 1.5/2.0 (-0.5)
Epoch 006/040 | Loss 0.0016 | Win/lose count 1.5/4.0 (-2.5)
Epoch 007/040 | Loss 0.0078 | Win/lose count 2.0/4.0 (-2.0)
Epoch 008/040 | Loss 0.0014 | Win/lose count 3.5/1.0 (2.5)
Epoch 009/040 | Loss 0.0037 | Win/lose count 5.0/2.0 (3.0)
Epoch 010/040 | Loss 0.0058 | Win/lose count 2.5/0 (2.5)
Epoch 011/040 | Loss 0.0076 | Win/lose count 4.0/3.0 (1.0)
Epoch 012/040 | Loss 0.0124 | Win/lose count 4.0/1.0 (3.0)
Epoch 013/040 | Loss 0.0064 | Win/lose count 3.0/2.0 (1.0)
Epoch 014/040 | Loss 0.0035 | Win/lose count 4.5/4.0 (0.5)
Epoch 015/040 | Loss 0.0049 | Win/lose count 4.5/4.0 (0.5)
Epoch 016/040 | Loss 0.0023 | Win/lose count 4.5/3.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [27]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 6.0/4.0. Average score (2.0)
Win/lose count 3.0/2.0. Average score (1.5)
Win/lose count 5.5/2.0. Average score (2.1666666666666665)
Win/lose count 8.5/3.0. Average score (3.0)
Win/lose count 6.5/2.0. Average score (3.3)
Win/lose count 4.0/2.0. Average score (3.0833333333333335)
Win/lose count 5.5/4.0. Average score (2.857142857142857)
Win/lose count 6.0/1.0. Average score (3.125)
Win/lose count 5.5/5.0. Average score (2.8333333333333335)
Win/lose count 4.5/3.0. Average score (2.7)
Win/lose count 3.5/2.0. Average score (2.590909090909091)
Win/lose count 1.5/2.0. Average score (2.3333333333333335)
Win/lose count 4.0/1.0. Average score (2.3846153846153846)
Win/lose count 5.0/2.0. Average score (2.4285714285714284)
Win/lose count 7.5/4.0. Average score (2.5)
Win/lose count 2.0/1.0. Average score (2.40625)
Win/lose count 3.0/3.0. Average score (2.264705882352941)
Win/lose count 3.5/0. Average score (2.3333333333333335)
Win/lose count 4.5/1.0. Average score (2.

In [28]:
HTML(display_videos('cnn_test10.mp4'))

In [20]:
HTML(display_videos('fc_test10.mp4'))

- The CNN architecture performs better than the fully-connected architecture. However, the agent doesn't seem to explore the map that much, and typically gets stuck between some states. 
- Effect of temperature: increasing temperature increases the positive rewards cells, which pushes the agent to explore more since positive reward is almost always in his reach. Thus, the final average score gets higher. However, When the temperature is low, the rat sometimes does not observe any positive reward and the problem of exploration is more influential. 



***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix='', eps_decay = 0.99):
    # Number of won games
    score = 0
    loss = 0
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
            #Decreasing eps-greedy policy
            agent.set_epsilon(agent.epsilon * eps_decay)
            
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')            
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.malus_position = np.zeros((grid_size,grid_size))
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        if train:
            reward -= self.malus_position[self.x, self.y]
            
        self.malus_position[self.x, self.y] = 0.1
        
        reward = reward + self.board[self.x, self.y]

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state       

In [30]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/040 | Loss 0.0203 | Win/lose count 8.0/25.10000000000006 (-17.10000000000006)
Epoch 001/040 | Loss 0.0173 | Win/lose count 6.0/19.700000000000017 (-13.700000000000017)
Epoch 002/040 | Loss 0.0048 | Win/lose count 3.5/19.300000000000004 (-15.800000000000004)
Epoch 003/040 | Loss 0.0096 | Win/lose count 15.0/16.299999999999972 (-1.2999999999999723)
Epoch 004/040 | Loss 0.0204 | Win/lose count 11.0/16.399999999999963 (-5.399999999999963)
Epoch 005/040 | Loss 0.0080 | Win/lose count 9.0/17.69999999999998 (-8.699999999999982)
Epoch 006/040 | Loss 0.0052 | Win/lose count 10.0/15.399999999999965 (-5.399999999999965)
Epoch 007/040 | Loss 0.0089 | Win/lose count 16.5/13.499999999999968 (3.000000000000032)
Epoch 008/040 | Loss 0.0087 | Win/lose count 13.5/15.499999999999964 (-1.9999999999999645)
Epoch 009/040 | Loss 0.0190 | Win/lose count 11.0/17.299999999999983 (-6.299999999999983)
Epoch 010/040 | Loss 0.0134 | Win/lose count 11.0/15.099999999999962 (-4.099999999999962)
Epoch 011/040

In [31]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 10.5/18.19999999999999. Average score (-7.699999999999989)
Win/lose count 23.5/14.099999999999973. Average score (0.8500000000000192)
Win/lose count 8.5/16.599999999999966. Average score (-2.1333333333333093)
Win/lose count 13.0/17.69999999999998. Average score (-2.7749999999999773)
Win/lose count 22.5/13.89999999999997. Average score (-0.49999999999997585)
Win/lose count 12.5/17.199999999999974. Average score (-1.1999999999999755)
Win/lose count 26.0/14.699999999999969. Average score (0.5857142857143111)
Win/lose count 11.0/16.89999999999997. Average score (-0.2249999999999741)
Win/lose count 11.0/17.899999999999984. Average score (-0.9666666666666419)
Win/lose count 5.5/17.899999999999984. Average score (-2.109999999999976)
Win/lose count 12.0/15.79999999999996. Average score (-2.2636363636363384)
Win/lose count 12.5/15.59999999999996. Average score (-2.333333333333307)
Win/lose count 10.0/15.89999999999996. Average score (-2.6076923076922798)
Win/lose count 12.0/15.19

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***